In [1]:
import os
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pickle

import openpyxl

In [2]:
output_path = os.path.join('C:\\','projects','pg_risk_analysis_output','output')
risk_path = os.path.join(output_path,'risk')
figure_path = os.path.join('C:\\','projects','pg_risk_analysis_output','output','figures')

# Read and restructure results

In [3]:
# Reframe FL OSM risk outputs
def restructure_output(hazard_type,infra_type):
    outputs = {}
    for file_name in os.listdir(risk_path):
        if file_name.endswith('.xlsx'):
            climate_model = os.path.splitext(file_name)[0].split('_')[3]
            if (hazard_type in file_name) & (infra_type in file_name) & (climate_model in file_name):
                check_sheets = pd.ExcelFile(os.path.join(risk_path,file_name),engine='openpyxl').sheet_names
                for sheet in check_sheets:
                    out = pd.read_excel(os.path.join(risk_path,file_name),sheet_name= sheet, index_col=[0],engine='openpyxl')
                    outputs[file_name[:3],climate_model,sheet] = out
    df = pd.concat(outputs).unstack(level=2).dropna(axis=1, how='all')
    
    # revise multiindex
    df.reset_index(inplace=True)
    df.columns = df.columns.droplevel(1)

    return df

In [4]:
df = restructure_output('tc','osm')
df#["level_0"].values

,level_0,level_1,level_2,W2_1_1,W2_1_2,W2_1_3,W2_1_4,W2_1_5,W2_1_6,W2_2_1,...,W4_47,W4_48,W4_49,W4_50,W4_51,W4_52,W4_53,W4_54,W4_55,W4_56
0,BRN,CMCC-CM2-VHR4,mean_risk,306119.530398,189113.843224,102039.843466,64285.101384,54081.117037,36394.210836,236290.977841,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BRN,CMCC-CM2-VHR4,lower_risk,229589.647799,141835.382418,76529.882600,48213.826038,40560.837778,27295.658127,177218.233381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BRN,CMCC-CM2-VHR4,upper_risk,382649.412998,236392.304030,127549.804333,80356.376730,67601.396296,45492.763545,295363.722301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BRN,CNRM-CM6-1-HR,mean_risk,327910.173326,202575.618188,109303.391109,68861.136398,57930.797288,38984.876162,259041.903720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BRN,CNRM-CM6-1-HR,lower_risk,245932.629994,151931.713641,81977.543331,51645.852299,43448.097966,29238.657122,194281.427790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,VNM,HadGEM3-GC31-HM,lower_risk,210461.189337,130018.245857,70153.729779,44196.849761,37181.476783,25021.496955,166014.867281,...,6322.530757,1931.254844,3139.281707,5011.144521,222024.807856,222313.262373,221328.268428,221696.667233,222195.197683,3650.931715
236,VNM,HadGEM3-GC31-HM,upper_risk,350768.648895,216697.076429,116922.882965,73661.416268,61969.127971,41702.494924,276691.445469,...,10537.551262,3218.758074,5232.136179,8351.907535,370041.346427,370522.103955,368880.447380,369494.445388,370325.329472,6084.886191
237,VNM,present,mean_risk,69648.962769,43027.581444,23216.320923,14626.282182,12304.650089,8280.487796,43839.443958,...,3192.800202,607.098053,1003.680387,1631.117963,296045.000404,296418.899046,295111.768537,295602.365005,296267.818266,2121.532542
238,VNM,present,lower_risk,52236.722077,32270.686083,17412.240692,10969.711636,9228.487567,6210.365847,32879.582968,...,2394.600152,455.323540,752.760290,1223.338472,222033.750303,222314.174284,221333.826402,221701.773754,222200.863700,1591.149407


# Create Table S1-S4

In [7]:
def analysis_results(hazard_type,infra_type,i):
    df = restructure_output(hazard_type,infra_type)
    df.rename(columns={df.columns[0]:'country_code',df.columns[1]:'climate_model',df.columns[2]:'risk_type'}, inplace=True)
    # df.fillna(0, inplace=True)
    
    df = df.loc[df['country_code'] != 'MNG']
            
    if hazard_type == 'tc':
        climate_models = ['present','CMCC-CM2-VHR4','CNRM-CM6-1-HR','EC-Earth3P-HR','HadGEM3-GC31-HM']
        code_list = ['W2','W3','W4','W5']
        
        if infra_type == 'osm':
            df = df.drop(['W3_1','W3_2','W3_3','W3_4','W3_5'],axis=1)
                
    elif hazard_type == 'fl':
        climate_models = ['historical','rcp8p5']
        code_list = ['F1','F2','F3','F4','F5']
    
    writer = pd.ExcelWriter(os.path.join(figure_path,f'Table_S{i}-{hazard_type}_{infra_type}_risk.xlsx'),engine='openpyxl')
    
    for climate_model in climate_models:
        for code in code_list:
            asset_select = df.columns[df.columns.str.startswith(code)]
            df_infra = df[(df['climate_model'] == climate_model) & (df['risk_type'] == 'mean_risk')][asset_select]
            
            # calculate total risk of each vul curve and min/max/median risk
            # Note: df_infra['median_risk'] should be inserted before appending the df_infra.sum(),
            # to keep the countries share the same vulnerability curve when calculating the sum risk
            
            # calculate min/max/median risk of each country
            df_infra.insert(0,'min_risk',df_infra.min(axis=1))
            df_infra.insert(1,'max_risk',df_infra.max(axis=1))
            df_infra.insert(2,'median_risk',df_infra.median(axis=1))
            
            sum_risk_row = pd.DataFrame([df_infra.sum()/10**6], columns=df_infra.columns)
            df_infra = pd.concat([df_infra,sum_risk_row], ignore_index=True)
            
            if infra_type == 'osm':
                df_infra.insert(0,'',['BRN','CHN','IDN','JPN','KHM','KOR','LAO','MMR','MYS','PHL','PRK','SGP','THA','TWN','VNM',
                                      'sum (million)'])
            elif infra_type == 'gov' and hazard_type == 'fl':
                df_infra.insert(0,'',['CHN','IDN','JPN','KHM','KOR','LAO','MMR','MYS','PHL','PRK','SGP','THA','TWN','VNM',
                                      'sum (million)'])
            elif infra_type == 'gov' and hazard_type == 'tc':
                df_infra.insert(0,'',['BRN','CHN','IDN','JPN','KHM','KOR','LAO','MMR','MYS','PHL','PRK','THA','TWN','VNM',
                                      'sum (million)'])
    
            df_infra.to_excel(writer,sheet_name=f'{climate_model}_{code}_risk', index=False)
        
    writer.close()

In [12]:
analysis_results('tc','osm',1)

In [9]:
analysis_results('tc','gov',2)

In [10]:
analysis_results('fl','osm',3)

In [11]:
analysis_results('fl','gov',4)